In [1]:
from util.ImgMapper import *
import pandas as pd
from PIL import Image
from sklearn.decomposition import PCA
from pylab import *
from io import BytesIO
from scipy.misc import imsave, toimage
import base64

ImportError: cannot import name inplace_column_scale

In [ ]:
df = pd.read_csv("../features/CUB/BVLCref_fc8.csv")
df.head()

In [ ]:
image_paths = list(df.iloc[:,0])
print len(image_paths)

In [ ]:
tooltip_s = []

for image_path in image_paths:
    output = BytesIO()
    #img = toimage(image_data.reshape((28,28))) # Data was a flat row of 64 "pixels".
    img = Image.open(image_path).resize((100,100))
    img.save(output, format="PNG")
    contents = "<img src='data:image/png;base64,"+base64.encodestring(output.getvalue()).decode()+"'>"
    tooltip_s.append( contents )
    output.close()

tooltip_s = np.array(tooltip_s) # need to make sure to feed it as a NumPy array, not a list

In [ ]:
image_features = array(df.iloc[:,1:])
image_features.shape

In [ ]:
### Reduce dimensions and apply clustering

pca = PCA(n_components=50)
data = pca.fit_transform(array(image_features))

# Initialize to use t-SNE with 2 components (reduces data to 2 dimensions). Also note high overlap_percentage.
mapper = KeplerMapper(cluster_algorithm=cluster.DBSCAN(eps=0.2, min_samples=5), 
             reducer = manifold.TSNE(), nr_cubes=20, overlap_perc=0.8, 
             link_local=False, verbose=1)

# Fit and transform data
data = mapper.fit_transform(data)

In [ ]:
data[:4,:]

In [ ]:
# Create the graph
complex = mapper.map(data, dimension_index=[0,1], dimension_name="t-SNE(2) 2D")

# Create the visualizations (increased the graph_gravity for a tighter graph-look.)

# Tooltips with image data for every cluster member
mapper.visualize(complex, "results-CUB-4.html", "t-SNE on CUB", graph_charge=-50, graph_gravity=0.15, custom_tooltips=tooltip_s)
  